In [90]:
import pandas as pd

# -- Selenium --
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from time import sleep
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException


# -- API --
import re
import json
import requests
from tqdm import tqdm

# Get today
from datetime import date

# Check time
from tqdm import tqdm

In [91]:
link_shops = pd.read_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/crawl_web_RA/20220824_sendo/sendo_recrawl_20220829/Link_shop_sendo.csv')


In [92]:
link_shops = link_shops.shop_url

In [94]:
def get_mail(driver):
    email = ''
    Xpaths = driver.find_elements(by=By.XPATH, value='//div[@class="panelHeading_3tNo custom1_1mgl"]')
    for xpath in Xpaths:
        if('Email' in xpath.text):
            email = email +  xpath.text.split('\n')[1]

    return email

def get_phone_number(driver):
    phone_number = ''
    Xpaths = driver.find_elements(by=By.XPATH, value='//div[@class="panelHeading_3tNo custom1_1mgl"]')
    for xpath in Xpaths:
        if('Điện thoại' in xpath.text):
            phone_number = phone_number + xpath.text.split('\n')[1]

    return phone_number

def get_website(driver):
    website = ''
    Xpaths = driver.find_elements(by=By.XPATH, value='//div[@class="panelHeading_3tNo custom1_1mgl"]')
    for xpath in Xpaths:
        if('Website' in xpath.text):
            website = website + xpath.text.split('\n')[1]

    return website

def get_response_rate(driver):
    Xpaths = driver.find_elements(by=By.XPATH, value='//div[@class="shop-info-block-item_1giu d7e-a90f22 d7e-57f266 d7e-6f98d6 d7e-f6de6c"]')
    response_rate = ''
    for xpath in Xpaths:
        if('Tỉ lệ phản hồi' in xpath.text):
            response_rate = response_rate + xpath.text.split('\n')[0]
    return response_rate

def get_num_of_product(driver):
    Xpaths = driver.find_elements(by=By.XPATH, value='//div[@class="shop-info-block-item_1giu d7e-a90f22 d7e-57f266 d7e-6f98d6 d7e-f6de6c"]')
    num_of_product = ''
    for xpath in Xpaths:
        if('Sản phẩm' in xpath.text):
            num_of_product = num_of_product+ xpath.text.split('\n')[0]
    return num_of_product

def get_location(driver):
    location = ''
    Xpaths = driver.find_elements(by=By.XPATH, value='//div[@class="panelHeading_3tNo custom1_1mgl"]')
    for xpath in Xpaths:
        if('Địa chỉ' in xpath.text):
            location = location + xpath.text.split('\n')[1]

    return location

def get_type_shop(driver):
    type_shop = []

    Xpath_check_type_shop_plus = driver.find_elements(by=By.XPATH, value='//div[@class="shop-summary-info__shop-type_3-dJ"]/img')
    if(len(Xpath_check_type_shop_plus) != 0):
        for type_shop_plus in Xpath_check_type_shop_plus:
            type_shop.append(type_shop_plus.get_attribute('src'))

    Xpath_check_type_shop_mall = driver.find_elements(by=By.XPATH, value='//div[@class="shop-summary-info__shop-type_3-dJ"]/div')
    if(len(Xpath_check_type_shop_mall) != 0):
        for type_shop_mall in Xpath_check_type_shop_mall:
            type_shop.append(type_shop_mall.get_attribute('class'))

    return type_shop

In [102]:
def startCrawl(product_links, list_product):
    driver = webdriver.Chrome(executable_path=r"C:/Users/Admin/OneDrive_Quan/Máy tính/crawl_web_RA/chromedriver/chromedriver")

    for link_shop in link_shops[100:]:
        try:
            try:
                driver.set_page_load_timeout(10)

                Product_Infos = {}

                driver.get(link_shop + '/thong-tin-shop')
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                sleep(0.5)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                sleep(2)

                Product_Infos['shop_url'] = link_shop
                if(len(driver.find_elements(by=By.XPATH, value='//span[@class="d7e-ed528f d7e-fde242 d7e-266ced d7e-fb1c84 undefined"]')) != 0):
                    Product_Infos['shop_name'] = driver.find_element(by=By.XPATH, value='//span[@class="d7e-ed528f d7e-fde242 d7e-266ced d7e-fb1c84 undefined"]').text
                else:
                    continue

                try:
                    Product_Infos['type_shop'] = get_type_shop(driver)
                except:
                    Product_Infos['type_shop'] = ''

                try:
                    Product_Infos['response_rate'] = get_response_rate(driver)
                except:
                    Product_Infos['response_rate'] = ''

                if(len(driver.find_elements(by=By.XPATH, value='//div[@class="panelHeading_3tNo custom1_1mgl"]')) != 0):
                    Product_Infos['joined_time'] = driver.find_element(by=By.XPATH, value='//div[@class="panelHeading_3tNo custom1_1mgl"]').text.split('\n')[1]
                else:
                    pass

                try:
                    Product_Infos['mail'] = get_mail(driver)
                    Product_Infos['phone_number'] = get_phone_number(driver)
                except:
                    Product_Infos['mail'] = ''
                    Product_Infos['phone_number'] = ''

                if(len(driver.find_elements(by=By.XPATH, value='//div[@class="slogan_3Zay"]')) != 0):
                    Product_Infos['description'] = driver.find_element(by=By.XPATH, value='//div[@class="slogan_3Zay"]').text
                else:
                    pass

                try:
                    Product_Infos['website'] = get_website(driver)
                except:
                    Product_Infos['website'] = ''

                if(len(driver.find_elements(by=By.XPATH, value='//span[@class="shop-rating__rating-number_3Uza d7e-d87aa1 d7e-fb1c84 d7e-a4f628"]')) != 0):
                    Product_Infos['avg_rating'] = driver.find_element(by=By.XPATH, value='//span[@class="shop-rating__rating-number_3Uza d7e-d87aa1 d7e-fb1c84 d7e-a4f628"]').text
                else:
                    pass

                if(len(driver.find_elements(by=By.XPATH, value='//span[@class="shop-rating__vote_3YoG undefined d7e-d87aa1 d7e-a58302"]')) != 0):
                    Product_Infos['total_rating_count'] = driver.find_element(by=By.XPATH, value='//span[@class="shop-rating__vote_3YoG undefined d7e-d87aa1 d7e-a58302"]').text[1:-1]
                else:
                    pass

                Product_Infos['platform'] = 'sendo'

                try:
                    Product_Infos['num_of_product'] = get_num_of_product(driver)
                except:
                    pass

                try:
                    Product_Infos['location'] = get_location(driver)
                except:
                    pass

                list_product.append(Product_Infos)
                print(len(list_product))
                sleep(0.5)

            except TimeoutException:
                print('time out')
                continue
        except:
            continue


In [103]:
import threading
import numpy as np

list_threads = []
list_product = []
num_thread = 3
list_partitions = [partition for partition in np.array_split(link_shops, num_thread)]
for partition_list in list_partitions:
    list_threads.append(threading.Thread(target=startCrawl, args=(partition_list, list_product)))

In [ ]:
for thread in list_threads:
    thread.start()
    time.sleep(5)

In [ ]:
for thread in list_threads:
    thread.join()

In [106]:
len(list_product)

48

In [107]:
pd.DataFrame(list_product).to_csv('C:/Users/Admin/OneDrive_Quan/Máy tính/crawl_web_RA/20220824_sendo/sendo_recrawl_20220829/shop_info_100__0.csv', index=False)